Check number of threads being used first

In [ ]:
num_threads = Threads.nthreads()

Code from Sparsification module

In [ ]:
using Distributions
using GeometricIntegrators
using Optim
using Random
using Flux
using Distances
using Symbolics
using ForwardDiff
using Plots
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

gr()

_prod(a, b, c, arrs...) = a .* _prod(b, c, arrs...)
_prod(a, b) = a .* b
_prod(a) = a

# generates a vector out of symbolic arrays (p,q) with a certain dimension
function get_z_vector(dims)
    @variables q[1:dims]
    @variables p[1:dims]
    z = vcat(q,p)
    return z
end

# make combinations of bases of just the order that is given 
# e.g order = 2 will give just the bases whose powers sum to 2
function poly_combos(z, order, inds...)
    if order == 0
        return Num[1]
    elseif order == length(inds)
        return [_prod([z[i] for i in inds]...)]
    else
        start_ind = length(inds) == 0 ? 1 : inds[end]
        return vcat([poly_combos(z, order, inds..., j) for j in start_ind:length(z)]...)
    end
end

# gives all bases monomials up to a certain order
function primal_monomial_basis(z, order::Int)
    return Vector{Symbolics.Num}(vcat([poly_combos(z, i) for i in 1:order]...))
end

# calculates coefficient bases up to a certain order
# mostly for use with trigonometric functions example sin(k*z),
# where k is the coefficient
function primal_coeff_basis(z, max_coeff::Int)
    return Vector{Symbolics.Num}(vcat([k .* z for k in 1:max_coeff]...))
end

# calculates +,-,*,/ between states as a new basis
# the return output is a set to avoid duplicates
function primal_operator_basis(z, operator)
    return Vector{Symbolics.Num}([operator(z[i], z[j]) for i in 1:length(z)-1 for j in i+1:length(z)] ∪ [operator(z[j], z[i]) for i in 1:length(z)-1 for j in i+1:length(z)])
end

function primal_power_basis(z, max_power::Int)
    if max_power > 0
        return Vector{Symbolics.Num}(vcat([z.^i for i in 1:max_power]...))
    elseif max_power < 0
        return Vector{Symbolics.Num}(vcat([z.^-i for i in 1:abs(max_power)]...))
    end
end

function polynomial_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = primal_monomial_basis(z, polyorder)
    primes = vcat(primes, primal_coeff_basis(z, max_coeff))
    if operator !== nothing
        primes = vcat(primes, primal_operator_basis(z, operator))
    end
    return primes
end

function trigonometric_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return vcat(sin.(primes), cos.(primes))
end

function exponential_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return exp.(primes)
end

function logarithmic_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return log.(abs.(primes))
end

function mixed_states_basis(basis::Vector{Symbolics.Num}...)
    mixed_states = Tuple(basis)
    
    ham = Vector{Symbolics.Num}()
    for i in eachindex(mixed_states)
        for j in i+1:lastindex(mixed_states)
            ham = vcat(ham, [mixed_states[i][k] * mixed_states[j][l] for k in 1:length(mixed_states[i]) for l in 1:length(mixed_states[j])])
        end
    end
    
    return Vector{Symbolics.Num}(ham)
end

# returns the number of required coefficients for the basis
function get_numCoeffs(basis::Vector{Symbolics.Num})
    return length(basis)
end


# gets a vector of combinations of hamiltonian basis
function get_basis_set(basis::Vector{Symbolics.Num}...)
    # gets a vector of combinations of basis
    basis = vcat(basis...)
    
    # removes duplicates
    basis = Vector{Symbolics.Num}(collect(unique(basis)))

    return basis
end

# returns a function that can build the gradient of the hamiltonian
function ΔH_func_builder(d::Int, z::Vector{Symbolics.Num} = get_z_vector(d), basis::Vector{Symbolics.Num}...) 
    # nd is the total number of dimensions of all the states, e.g. if q,p each of 3 dims, that is 6 dims in total
    nd = 2d
    Dz = Differential.(z)
    
    # collects and sums combinations of basis and coefficients"
    basis = get_basis_set(basis...)
   
    # gets number of terms in the basis
    @variables a[1:get_numCoeffs(basis)]
    
    # collect and sum combinations of basis and coefficients
    ham = sum(collect(a .* basis))
    
    # gives derivative of the hamiltonian, but not the skew-symmetric true one
    f = [expand_derivatives(dz(ham)) for dz in Dz]

    #simplify the expression potentially to make it faster
    f = simplify(f)
    
    # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
    ∇H = vcat(f[d+1:2d], -f[1:d])
    
    # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
    ∇H_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(∇H, z, a)[2]))
    
    return ∇H_eval
end

struct HamiltonianSINDy{T, GHT}
    basis::Vector{Symbolics.Num} # the augmented basis for sparsification
    analytical_fθ::GHT
    z::Vector{Symbolics.Num} 
    λ::T # Sparsification Parameter
    noise_level::T # Noise amplitude added to the data
    noiseGen_timeStep::T # Time step for the integrator to get noisy data 
    nloops::Int # Sparsification Loops
    
    function HamiltonianSINDy(basis::Vector{Symbolics.Num},
        analytical_fθ::GHT = missing,
        z::Vector{Symbolics.Num} = get_z_vector(2);
        λ::T = 0.05,
        noise_level::T = 0.00,
        noiseGen_timeStep::T = 0.05,
        nloops = 10) where {T, GHT <: Union{Base.Callable,Missing}}

        new{T, GHT}(basis, analytical_fθ, z, λ, noise_level, noiseGen_timeStep, nloops)
    end
end

function gen_noisy_ref_data(method::HamiltonianSINDy, x)
    # initialize timestep data for analytical solution
    tstep = method.noiseGen_timeStep
    tspan = (zero(tstep), tstep)

    function next_timestep(x)
        prob_ref = ODEProblem((dx, t, x, params) -> method.analytical_fθ(dx, x, params, t), tspan, tstep, x)
        sol = integrate(prob_ref, Gauss(2))
        sol.q[end]
    end

    data_ref = [next_timestep(_x) for _x in x]

    # add noise
    data_ref_noisy = [_x .+ method.noise_level .* randn(size(_x)) for _x in data_ref]

    return data_ref_noisy

end

struct TrainingData{AT<:AbstractArray}
    x::AT # initial condition
    ẋ::AT # initial condition
    y::AT # noisy data at next time step

    TrainingData(x::AT, ẋ::AT, y::AT) where {AT} = new{AT}(x, ẋ, y)
    TrainingData(x::AT, ẋ::AT) where {AT} = new{AT}(x, ẋ)
end



In [20]:
# --------------------
# Setup
# --------------------

println("Setting up...")

# 2D system with 4 variables [q₁, q₂, p₁, p₂]
const nd = 4

z = get_z_vector(nd/2)
polynomial = polynomial_basis(z, polyorder=3)
trigonometric  = trigonometric_basis(z, max_coeff=1)
prime_diff = primal_operator_basis(z, -)
basis = get_basis_set(polynomial, trigonometric, prime_diff)
# initialize analytical function, keep λ smaller than ϵ so system is identifiable
ϵ = 0.5
m = 1

# two-dim simple harmonic oscillator (not used anywhere only in case some testing needed)
# H_ana(x, p, t) = ϵ * x[1]^2 + ϵ * x[2]^2 + 1/(2*m) * x[3]^2 + 1/(2*m) * x[4]^2
# H_ana(x, p, t) = cos(x[1]) + cos(x[2]) + 1/(2*m) * x[3]^2 + 1/(2*m) * x[4]^2

# Gradient function of the 2D hamiltonian
# grad_H_ana(x) = [x[3]; x[4]; -2ϵ * x[1]; -2ϵ * x[2]]
grad_H_ana(x) = [x[3]; x[4]; sin(x[1]); sin(x[2])]
function grad_H_ana!(dx, x, p, t)
    dx .= grad_H_ana(x)
end
# ------------------------------------------------------------
# Training Data
# ------------------------------------------------------------

println("Generate Training Data...")

# number of samples
num_samp = 18

# samples in p and q space
samp_range = LinRange(-20, 20, num_samp)

# initialize vector of matrices to store ODE solve output

# s depend on size of nd (total dims), 4 in the case here so we use samp_range x samp_range x samp_range x samp_range
s = collect(Iterators.product(fill(samp_range, nd)...))


# compute vector field from x state values
x = [collect(s[i]) for i in eachindex(s)]
dx = zeros(nd)
p = 0
t = 0
ẋ = [grad_H_ana!(copy(dx), _x, p, t) for _x in x]


# ----------------------------------------
# Compute Sparse Regression
# ----------------------------------------

# choose SINDy method
# (λ parameter must be close to noise value so that only coeffs with value around the noise are sparsified away)
# noiseGen_timeStep chosen randomly for now
method = HamiltonianSINDy(basis, grad_H_ana!, z, λ = 0.05, noise_level = 0.0, noiseGen_timeStep = 0.05) #noise_level = 0.6

# generate noisy references data at next time step
y = gen_noisy_ref_data(method, x)

# collect training data
tdata = TrainingData(x, ẋ, y)

Setting up...
Generate Training Data...

TrainingData{Vector{Vector{Float64}}}([[-20.0, -20.0, -20.0, -20.0], [-17.647058823529413, -20.0, -20.0, -20.0], [-15.294117647058824, -20.0, -20.0, -20.0], [-12.941176470588234, -20.0, -20.0, -20.0], [-10.588235294117645, -20.0, -20.0, -20.0], [-8.235294117647058, -20.0, -20.0, -20.0], [-5.882352941176468, -20.0, -20.0, -20.0], [-3.529411764705884, -20.0, -20.0, -20.0], [-1.1764705882352935, -20.0, -20.0, -20.0], [1.1764705882352935, -20.0, -20.0, -20.0]  …  [-1.1764705882352935, 20.0, 20.0, 20.0], [1.1764705882352935, 20.0, 20.0, 20.0], [3.529411764705884, 20.0, 20.0, 20.0], [5.882352941176471, 20.0, 20.0, 20.0], [8.235294117647062, 20.0, 20.0, 20.0], [10.588235294117645, 20.0, 20.0, 20.0], [12.941176470588232, 20.0, 20.0, 20.0], [15.294117647058824, 20.0, 20.0, 20.0], [17.647058823529413, 20.0, 20.0, 20.0], [20.0, 20.0, 20.0, 20.0]], [[-20.0, -20.0, -0.9129452507276277, -0.9129452507276277], [-20.0, -20.0, 0.9329410219916074, -0.9129452507276277], [-20.0, -20.0, -0.40213327081314787

In [ ]:
# dimension of system
d = size(tdata.x[begin], 1) ÷ 2

# returns function that builds hamiltonian gradient through symbolics
fθ = ΔH_func_builder(d, method.z, method.basis)

In [21]:

# evaluate neural network
function network_two(xᵢₙ, model, method, fθ)
	# input layer
	# input_layer = relu.(model[1].W  * xᵢₙ)

	# first hidden layer
	# x₀ = relu.(model[2].W * input_layer .+ model[2].b)
    x₀ = leakyrelu.(model[1].W  * xᵢₙ .+ model[1].b, 0.05) #rrelu
	# SINDy layer
	function SINDy_layer()
        # coeffs initialized to a vector of zeros b/c easier to optimize zeros for our case
        coeffs = model[2].W

        numLoops = 4 # random choice of loop steps
        
        local x̄ = zeros(eltype(coeffs), axes(x₀))
        local x̃ = zeros(eltype(coeffs), axes(x₀))
        local f = zeros(eltype(coeffs), axes(x₀))

        # gradient at current (x) values
        fθ(f, x₀, coeffs)

        # for first guess use explicit euler
        x̃ .= x₀ .+ method.noiseGen_timeStep .* f
        
        for _ in 1:numLoops
            x̄ .= (x₀ .+ x̃) ./ 2
            # find gradient at {(x̃ₙ + x̃ⁱₙ₊₁)/2} to get Hermite extrapolation
            fθ(f, x̄, coeffs)
            # mid point rule for integration to next step
            x̃ .= x₀ .+ method.noiseGen_timeStep .* f
        end
        return x̃
    end
    # return  SINDy_layer()
	# output layer (linear activation)
	return (model[3].W * SINDy_layer())
end

network_two (generic function with 1 method)

In [ ]:
function loss_kernel(xᵢₙ, x₁, model, method, fθ)
    x̃₁ = network_two(xᵢₙ, model, method, fθ)
    # calculate square Euclidean distance
    sqeuclidean(x̃₁, x₁)
end

In [ ]:
# define loss function
function loss(flattened_model::AbstractVector)
    # λ_l2 = 0.01  # L2 regularization strength
    # λ_l1 = 0.01  # L1 regularization strength
    # reg_loss = λ_l2 * sum(abs2, flattened_model) + λ_l1 * sum(abs, flattened_model)
    # Convert the flattened parameters back to the original structure
    local recon_model = reconstruct_model(flattened_model, ld, ndim)
    # return reg_loss + mapreduce(z -> loss_kernel(z..., recon_model, method, fθ), +, zip(tdata.x, tdata.y)) 
    return mapreduce(z -> loss_kernel(z..., recon_model, method, fθ), +, zip(tdata.x, tdata.y)) 
end

In [ ]:
function flatten_model(model)
    θ = [model[1].W, model[1].b, model[2].W, model[3].W]
    # Flatten the model into a single vector
    return flattened_model = cat([vec(θ[i]) for i in 1:length(θ)]..., dims=1)
end

In [ ]:
function reconstruct_model(flattened_model,ld,ndim)
    reconstructed_model = (
        (W = reshape(flattened_model[1:ld*ndim], ld, ndim), b = reshape(flattened_model[(ld*ndim)+1:(ld*ndim)+ld], ld)),
        (W = flattened_model[(ld*ndim)+ld+1:(ld*ndim)+ld+get_numCoeffs(method.basis)], ),
        (W = reshape(flattened_model[(ld*ndim)+ld+get_numCoeffs(method.basis)+1:end], ndim, ld), ),
    )
    return reconstructed_model
end

In [22]:
using LinearAlgebra
# latent dimension: ld
ld = size(tdata.x[begin], 1)
ndim = size(tdata.x[begin], 1)
model = (
	(W = Matrix{Float64}(I, ld, ndim), b = zeros(ld)),   #(W = randn(ld, ndim), b = randn(ld)),
	(W = zeros(get_numCoeffs(method.basis)), ),
	(W = Matrix{Float64}(I, ndim, ld), ) #(W = randn(ndim, ld), ),
)

((W = [1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0], b = [0.0, 0.0, 0.0, 0.0]), (W = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],), (W = [1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0],))

In [ ]:
# initial_loss_array = Vector{Float64}()

# # initial guess
# println("Initial Guess...")
# flattened_model = flatten_model(model)
# # Define the optimization solver
# solver = BFGS()
# result = Optim.optimize(loss, flattened_model, solver, Optim.Options(show_trace=true); autodiff = :forward)
# @show loss(flattened_model)
# push!(initial_loss_array, loss(flattened_model))

# reconstructed_model = reconstruct_model(result.minimizer, ld, ndim)

# coeffs = reconstructed_model[2].W

# (result)

In [23]:
initial_loss_array = Vector{Float64}()

# initial guess
println("Initial Guess...")

flattened_model = flatten_model(model)
@show reconstruct_model(flattened_model, ld, ndim)[2].W

optimizer = AMSGrad()#AdamW()

for i in 1:60
    # Compute gradients using ForwardDiff.jl
    gradients = ForwardDiff.gradient(loss, flattened_model)

    # Update the parameters using the optimizer
    Flux.Optimise.update!(optimizer, flattened_model, gradients)
    push!(initial_loss_array, loss(flattened_model))
    @show i, loss(flattened_model)
    @show reconstruct_model(flattened_model, ld, ndim)[2].W
end

Initial Guess...
((reconstruct_model(flattened_model, ld, ndim))[2]).W = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


(i, loss(flattened_model)) = (1, 2.782880207830313e7)
((reconstruct_model(flattened_model, ld, ndim))[2]).W = [0.003162277660144041, 0.0031622776601440404, -0.0031622776601548217, -0.0031622776601548225, 0.0031622776601659564, 0.0031622776601659577, -0.0031622776284495296, -8.730579757534373e-8, 0.003162277660165956, -8.953324221477148e-8, -0.0031622776284495296, -0.003162277660166784, -0.00316227766016703, -0.0031622776601667834, 0.0031622776601682684, 0.0031622776601681834, -0.0031622776601666156, -0.003162277660167816, 0.0031622776601681825, 0.0031622776601674153, 0.0031622776601674158, 0.003162277660166803, -0.0031622776601678416, 0.0031622776601676157, 0.003162277660168269, -0.003162277660167816, -0.0031622776601666156, 0.0031622776601676157, -0.003162277660167842, 0.0031622776601668025, -0.0031622776601683052, -0.003162277660168262, -0.003162277660168262, -0.0031622776601683052, 0.0031622776601122815, 0.0031622776601122815, -0.0031622776601394165, -0.003162277660139416, 0.0031622


(i, loss(flattened_model)) = (2, 2.7199848951465715e7)
((reconstruct_model(flattened_model, ld, ndim))[2]).W = [0.007408453603005455, 0.007408453603015612, -0.007408278192401519, -0.0074082781924061075, 0.007406463576585843, 0.007406848435686307, -0.00015102232871083698, 0.003162190351848481, 0.007406463576609946, 0.003162188124391338, -0.00015102231809578084, -0.007404841540703518, -0.007406748262960502, -0.007404841540727163, 0.007405893257818845, 0.007405464464691378, -0.007401466127606603, -0.007410659641195426, 0.007405464464194832, 0.007402838690399171, 0.007402838689746017, 0.0074110687514692825, -0.007401495423021355, 0.007411868666930553, 0.007405893257843247, -0.007410659640066992, -0.007401466127145006, 0.007411868664240476, -0.007401495425502385, 0.007411068751559634, -0.007404178244711495, -0.007404827957662775, -0.007404827957410426, -0.007404178244734112, 0.007407837572774234, 0.007407837572810807, -0.0074076495680381275, -0.007407649568053008, 0.00741291179190277, 0.00


(i, loss(flattened_model)) = (3, 2.650674928551151e7)
((reconstruct_model(flattened_model, ld, ndim))[2]).W = [0.01234575036381865, 0.012345750326682964, -0.012345364113133278, -0.012345364096368574, 0.01233241853309117, 0.012337631594880726, 0.004012344044026266, 0.005998646961147538, 0.012332418478428626, 0.005998434939174112, 0.004012341206681038, -0.012331998517128635, -0.012337783630398672, -0.012331998470295378, 0.01232922870958654, 0.012330468966339587, -0.012353752852333696, -0.01236478598860299, 0.01233046897007262, 0.012302659058642258, 0.01230265903261359, 0.012355436407462929, -0.012308956225367885, 0.012360027595567724, 0.012329228655683817, -0.012364786040474529, -0.012353752925241688, 0.012360027507560139, -0.012308956266630085, 0.012355436315607631, -0.012329810729069546, -0.01232860674739969, -0.012328606740971424, -0.012329810691857328, 0.01234456573994001, 0.012344565685743662, -0.012340477981724544, -0.012340477916778562, 0.012368259573826151, 0.01236825959910913, 

(i, loss(flattened_model)) = (4, 2.5795710069570545e7)
((reconstruct_model(flattened_model, ld, ndim))[2]).W = [0.017756976439167585, 0.01775701588683826, -0.017756481828334465, -0.017756513910604222, 0.017711860596049553, 0.017733927474975514, 0.008854406347681896, 0.009333573065823837, 0.017711929415019593, 0.009469782638872383, 0.008859063881187132, -0.01772168149872703, -0.017732775255282735, -0.01772175039328596, 0.017701357164498423, 0.017708283133221966, -0.017819692832407603, -0.017814801797066897, 0.017708279937334023, 0.017628893946560354, 0.017628891269406425, 0.01778002313675114, -0.017634935099224425, 0.017788131013540508, 0.017701423231177557, -0.01781471424074182, -0.017819688918820797, 0.017788239649532613, -0.017634909696962277, 0.017780072720076274, -0.017716194289275056, -0.01770737963871376, -0.017707381010010386, -0.017716254471320088, 0.01775667539757096, 0.01775674653511422, -0.0177409056912321, -0.017740986133610897, 0.017821861341500305, 0.01782180377746749, -0

In [ ]:
# initial_loss_array = Vector{Float64}()
# initial_i = Vector{Float64}()
# initial_j = Vector{Float64}()

# # initial guess
# println("Initial Guess...")

# flattened_model = flatten_model(model)
# @show reconstruct_model(flattened_model, ld, ndim)[2].W

# optimizer = NAdam()

# batch_size = 1000  # Set your desired batch size

# num_batches = ceil(Int, length(tdata.x) / batch_size)

# for i in 1:50
#     for j in 1:num_batches
#         # Get the indices for the current batch
#         batch_indices = (j-1)*batch_size+1:min(j*batch_size, length(tdata.x))
        
#         # Extract the current batch from tdata.x and tdata.y
#         batch_x = tdata.x[batch_indices]
#         batch_y = tdata.y[batch_indices]
        
#         # Define the loss function for the current batch
#         batch_loss(flattened_model::AbstractVector) = begin
#             local recon_model = reconstruct_model(flattened_model, ld, ndim)
#             mapreduce(z -> loss_kernel(z..., recon_model, method, fθ), +, zip(batch_x, batch_y))
#         end
        
#         # Compute gradients using ForwardDiff.jl
#         gradients = ForwardDiff.gradient(batch_loss, flattened_model)

#         # Update the parameters using the optimizer
#         Flux.Optimise.update!(optimizer, flattened_model, gradients)
        
#         push!(initial_loss_array, batch_loss(flattened_model))
#         push!(initial_i, i)
#         push!(initial_j, j)
#         if i % 2 == 0 && j % 20 == 0
#             @show i, j, batch_loss(flattened_model)
#             @show reconstruct_model(flattened_model, ld, ndim)[2].W
#         end
#     end
# end


In [ ]:
q0 = plot(initial_loss_array, label = "initial batch loss")
# plot!(q0, log.(initial_i), label = "i index")
# plot!(q0, log.(initial_j), label = "j index")

In [ ]:
reconstructed_model = reconstruct_model(flattened_model, ld, ndim)
coeffs = reconstructed_model[2].W
println(coeffs)

In [ ]:
sparse_loss_array = Vector{Float64}()

for n in 1:method.nloops
    println("Iteration #$n...")

    # find coefficients below λ threshold
    smallinds = abs.(coeffs) .< method.λ
    biginds = .~smallinds
    
    # check if there are any small coefficients != 0 left
    #TODO: is the code expected to exit the loop here usually?
    all(coeffs[smallinds] .== 0) && break

    # set all small coefficients to zero
    coeffs[smallinds] .= 0

    # Regress dynamics onto remaining terms to find sparse coeffs
    function sparseloss(flattened_model::AbstractVector)
        c = zeros(eltype(flattened_model), axes(coeffs))
        c[biginds] .= flattened_model[(ld*ndim)+ld+1:end - ld*ndim]

        local reconstructed_model = ((W = reshape(flattened_model[1:ld*ndim], ld, ndim), b = reshape(flattened_model[(ld*ndim)+1:(ld*ndim)+ld], ld)),
            (W = c, ),
            (W = reshape(flattened_model[end - ld*ndim + 1:end], ndim, ld), ),
        )
        return loss(flatten_model(reconstructed_model))
    end

    # θ is partly a reference to coeffs[biginds] so coeffs[biginds] will be updated
    θ = [reconstructed_model[1].W, reconstructed_model[1].b, coeffs[biginds], reconstructed_model[3].W]
    # Flatten the model into a single vector
    flattened_model = cat([vec(θ[i]) for i in 1:length(θ)]..., dims=1)
    
    for i in 1:600
        # Compute gradients using ForwardDiff.jl
        gradients = ForwardDiff.gradient(sparseloss, flattened_model)
    
        # Update the parameters using the optimizer
        Flux.Optimise.update!(optimizer, flattened_model, gradients)
        push!(sparse_loss_array, sparseloss(flattened_model))
        @show i, sparseloss(flattened_model)
    end

    coeffs[biginds] = flattened_model[(ld*ndim)+ld+1:end - ld*ndim]

    reconstructed_model = ((W = reshape(flattened_model[1:ld*ndim], ld, ndim), b = reshape(flattened_model[(ld*ndim)+1:(ld*ndim)+ld], ld)),
            (W = coeffs, ),
            (W = reshape(flattened_model[end - ld*ndim + 1:end], ndim, ld), ),
        )
    @show coeffs
end


In [ ]:
# optimizer = ADAM(0.000005)
# sparse_loss_array = Vector{Float64}()
# sparse_i = Vector{Float64}()
# sparse_j = Vector{Float64}()


# for n in 1:method.nloops
#     println("Iteration #$n...")

#     # find coefficients below λ threshold
#     smallinds = abs.(coeffs) .< method.λ
#     biginds = .~smallinds
    
#     # check if there are any small coefficients != 0 left
#     #TODO: is the code expected to exit the loop here usually?
#     all(coeffs[smallinds] .== 0) && break

#     # set all small coefficients to zero
#     coeffs[smallinds] .= 0

#     # Regress dynamics onto remaining terms to find sparse coeffs
#     function sparseloss(flattened_model::AbstractVector, batch_x, batch_y)
#         c = zeros(eltype(flattened_model), axes(coeffs))
#         c[biginds] .= flattened_model[(ld*ndim)+ld+1:end - ld*ndim]

#         local reconstructed_model = ((W = reshape(flattened_model[1:ld*ndim], ld, ndim), b = reshape(flattened_model[(ld*ndim)+1:(ld*ndim)+ld], ld)),
#             (W = c, ),
#             (W = reshape(flattened_model[end - ld*ndim + 1:end], ndim, ld), ),
#         )
#         # Define the loss function for the current batch
#         batch_loss(flattened_model::AbstractVector) = begin
#             local recon_model = reconstruct_model(flattened_model, ld, ndim)
#             mapreduce(z -> loss_kernel(z..., recon_model, method, fθ), +, zip(batch_x, batch_y))
#         end
#         return batch_loss(flatten_model(reconstructed_model))
#     end

#     # θ is partly a reference to coeffs[biginds] so coeffs[biginds] will be updated
#     θ = [reconstructed_model[1].W, reconstructed_model[1].b, coeffs[biginds], reconstructed_model[3].W]
#     # Flatten the model into a single vector
#     flattened_model = cat([vec(θ[i]) for i in 1:length(θ)]..., dims=1)
    
#     for i in 1:100
#         for j in 1:num_batches
#             # Get the indices for the current batch
#             batch_indices = (j-1)*batch_size+1:min(j*batch_size, length(tdata.x))
            
#             # Extract the current batch from tdata.x and tdata.y
#             batch_x = tdata.x[batch_indices]
#             batch_y = tdata.y[batch_indices]
            
#             # Compute gradients using ForwardDiff.jl
#             gradients = ForwardDiff.gradient(flattened_model -> sparseloss(flattened_model, batch_x, batch_y), flattened_model)
        
#             # Update the parameters using the optimizer
#             Flux.Optimise.update!(optimizer, flattened_model, gradients)
            
#             if i % 10 == 0 && j % 100 == 0
#                 push!(sparse_loss_array, sparseloss(flattened_model, batch_x, batch_y))
#                 push!(sparse_i, i)
#                 push!(sparse_j, j)
#                 @show sparseloss(flattened_model, batch_x, batch_y)
#             end
#         end
#     end

#     coeffs[biginds] = flattened_model[(ld*ndim)+ld+1:end - ld*ndim]

#     reconstructed_model = ((W = reshape(flattened_model[1:ld*ndim], ld, ndim), b = reshape(flattened_model[(ld*ndim)+1:(ld*ndim)+ld], ld)),
#             (W = coeffs, ),
#             (W = reshape(flattened_model[end - ld*ndim + 1:end], ndim, ld), ),
#         )
#     @show coeffs
# end

In [ ]:
p0 = plot(log.(sparse_loss_array))
# plot!(p0, log.(sparse_i), label = "i index")
# plot!(p0, log.(sparse_j), label = "j index")

In [ ]:
reconstructed_model

In [ ]:
coeffs = reconstructed_model[2].W

In [ ]:
function (vectorfield)(dz, z)
    fθ(dz, z, reconstructed_model[2].W)
    return dz
end

(vectorfield)(dz, z, p, t) = vectorfield(dz, z)

In [ ]:

# ----------------------------------------
# Plot Results
# ----------------------------------------

println("Plotting...")

tstep = 0.01
tspan = (0.0,25.0)

for i in 1:5
    idx = rand(1:length(s))

    prob_reference = ODEProblem((dx, t, x, params) -> grad_H_ana!(dx, x, params, t), tspan, tstep, x[idx])
    data_reference = integrate(prob_reference, Gauss(1))

    prob_sindy = ODEProblem((dx, t, x, params) -> vectorfield(dx, x, params, t), tspan, tstep, x[idx])
    data_sindy = integrate(prob_sindy, Gauss(1))

    p1 = plot(xlabel = "Time", ylabel = "q₁")
    scatter!(p1, data_reference.t, data_reference.q[:,1], label = "Data q₁")
    scatter!(p1, data_sindy.t, data_sindy.q[:,1], markershape=:xcross, label = "Identified q₁")

    p3 = plot(xlabel = "Time", ylabel = "p₁")
    scatter!(p3, data_reference.t, data_reference.q[:,3], label = "Data p₁")
    scatter!(p3, data_sindy.t, data_sindy.q[:,3], markershape=:xcross, label = "Identified p₁")

    plot!(size=(1000,1000))
    display(plot(p1, p3, title="Analytical vs Calculated q₁ & p₁ in a 2D system with Euler"))

    p2 = plot(xlabel = "Time", ylabel = "q₂")
    scatter!(p2, data_reference.t, data_reference.q[:,2], label = "Data q₂")
    scatter!(p2, data_sindy.t, data_sindy.q[:,2], markershape=:xcross, label = "Identified q₂")

    p4 = plot(xlabel = "Time", ylabel = "p₂")
    scatter!(p4, data_reference.t, data_reference.q[:,4], label = "Data p₂")
    scatter!(p4, data_sindy.t, data_sindy.q[:,4], markershape=:xcross, label = "Identified p₂")

    plot!(size=(1000,1000))
    display(plot(p2, p4, title="Analytical vs Calculated q₂ & p₂ in a 2D system with Euler"))

end